# Panel Data 
Los modelos de panel data o datos de   son modelos que trabajan con data tomada a n individuos a lo largo de t periodos. Se distinguen dos casos de estos modelos basados en las dimensiones de la data disponible: 
1. Cuando t >> n, es decir, que la dimensión temporal es mucho mayor que la dimensión de la sección cruzada. A estos tipos de panel data se les conoce como panel de serie de tiempo, se los encuentra a menudo en macroeconomía y son los menos comunes. 
2. Cuando t << n, es decir, que la dimension de la sección cruzada es mucho mayor que la dimensión temporal. A estos tipos de panel data se les conoce como panel de sección cruzada, se los encuentra a menudo en microeconomía y son los más comunes. En general, los modelos de panel data están más orientados al análisis de este tipo de data. 
3. Cuando ti != t, es decir, que la dimension temporal varia para cada individuo. A estos tipos de panel data se les conoce como panel desbalanceado. 

Las fórmulas que se desarrollan en las siguientes secciones asumen que la data proviene de un panel de datos balanceados; sin embargo, la biblioteca de Python con la que trabajaremos Linearmodels permite estimar modelos utilizando paneles de datos desbalanceados. 

En general, los modelos de panel data tienen las siguientes ventajas: 
* Como se dispone de un mayor número de observaciones nt, se reduce la colinealidad entre las variables exógenas lo que mejora la eficiencia de los coeficientes. 
* Nos permite capturar la heterogeneidad no observable, ya sea entre individuos como a lo largo del tiempo. Además, permite aplicar una pruebas de hipótesis para confirmar o rechazar dicha heterogeneidad. 
* Los modelos de panel data son más informativos ya que reflejan la dinámica y causalidad de Granger a lo largo de todas las variables. 
* Permite diseñar y probar modelos complejos que buscan explicar el comportamiento de varios individuos a lo largo del tiempo. 
Por otra parte, las desventajas de este modelo están asociadas principalmente a los procesos para la obtención y el procesamiento de la data de los individuos. Entre estás desventajas tenemos: 
* Es muy probable que las observaciones no sean independientes. 
* Las muestras utilizadas pueden no ser representativas.


Primero vamos a definir una especificación general de un modelo de panel data ( n individuos a lo largo de t periodos):

![Panel](img/Panel_Data_0.png "Modelo General")

En los modelos de datos de panel, el objetivo principal es estimar los efectos parciales β. A partir del modelo general, vamos a desarrollar diversos modelos en donde se asumen diferentes tipos de efectos individuales y temporales.

## 12.3. Regresión Agrupada (Pooled Regression)

Este modelo se obtiene cuando se asume que Zi,j solo contiene un término constante.

### 12.3.1 Mínimos Cuadrados Ordinarios (MCO u OLS)

Este caso consiste en utilizar el estimador MCO. 

In [3]:
############################################################ 
# Importando el Dataset 
############################################################ 
import numpy as np 
import pandas as pd 
columns = ['EXP', 'WKS', 'OCC', 'IND', 'SOUTH', 'SMSA', 'MS', 'FEM', 'UNION', 'ED', 'BLK', 'LWAGE'] 
url = 'http://www.stern.nyu.edu/~wgreene/Text/Edition7/TableF8-1.csv' 
data = pd.read_csv(url, header=0) 
data.columns = columns 

# Cornwell y Rupert (1988) usan información de 595 individuos durante 7 años entre 1972-1982 
#
# Crea tuples (individual, year) to initialize index
#
tuples = [] 
for i in range(1,596):   # 595 individuos
    for j in range(1,8): # 7 años
        tuples.append((i,j)) 
index = pd.MultiIndex.from_tuples(tuples, names=['Individual', 'Year']) 

data.index = index 
display(data.head())


EXP  WKS  OCC  IND  SOUTH  SMSA  MS  FEM  UNION  ED  BLK  \
Individual Year                                                             
1          1       3   32    0    0      1     0   1    0      0   9    0   
           2       4   43    0    0      1     0   1    0      0   9    0   
           3       5   40    0    0      1     0   1    0      0   9    0   
           4       6   39    0    0      1     0   1    0      0   9    0   
           5       7   42    0    1      1     0   1    0      0   9    0   

                   LWAGE  
Individual Year           
1          1     5.56068  
           2     5.72031  
           3     5.99645  
           4     5.99645  
           5     6.06146

In [13]:
data.shape

(4165, 12)

#### DOS METODOS

#### A.- LINEARMODELS

In [4]:
############################################################ 
#Calculando los coeficientes usando MCO y Linearmodels 
############################################################ 
import linearmodels as lm 
import statsmodels.api as sm 
#Definiendo las variables 
exog = ['EXP', 'EXP^2', 'WKS', 'OCC', 'IND', 'SOUTH', 'SMSA', 'MS', 'FEM', 'UNION', 'ED', 'BLK'] 
data['EXP^2'] = data['EXP'] ** 2
Y = data['LWAGE']
X = sm.add_constant(data[exog]) 
#Estimando el modelo 
res_pooled = lm.panel.PooledOLS(Y, X).fit() 
print(res_pooled)

                          PooledOLS Estimation Summary                          
Dep. Variable:                  LWAGE   R-squared:                        0.4286
Estimator:                  PooledOLS   R-squared (Between):              0.5324
No. Observations:                4165   R-squared (Within):               0.1499
Date:                Fri, Apr 15 2022   R-squared (Overall):              0.4286
Time:                        14:52:40   Log-likelihood                   -1523.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      259.54
Entities:                         595   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                 F(12,4152)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             259.54
                            

#### B.- STATSMODELS

In [5]:
############################################################ 
#Calculando los coeficientes usando MCO y Statsmodels 
############################################################ 
#Estimando el modelo 
res_pooled_2 = sm.OLS(Y, X).fit() 
print(res_pooled_2.summary())




                            OLS Regression Results                            
Dep. Variable:                  LWAGE   R-squared:                       0.429
Model:                            OLS   Adj. R-squared:                  0.427
Method:                 Least Squares   F-statistic:                     259.5
Date:                Fri, 15 Apr 2022   Prob (F-statistic):               0.00
Time:                        14:52:44   Log-Likelihood:                -1523.3
No. Observations:                4165   AIC:                             3073.
Df Residuals:                    4152   BIC:                             3155.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.2511      0.071     73.662      0.0

### 12.3.2. ESTIMADOR ENTRE GRUPOS (BETWEEN OLS) - DOS METODOS

Este caso consiste en utilizar el estimador de MCO sobre los promedios de las variables a lo largo del tiempo.

#### A.- LINEARMODELS

In [6]:
################################################################ 
# Calculando los coeficientes usando Between OLS y Linearmodels 
################################################################ 
#Estimando el modelo 
res_between = lm.panel.BetweenOLS(Y, X).fit() 
print(res_between)


                         BetweenOLS Estimation Summary                          
Dep. Variable:                  LWAGE   R-squared:                        0.5443
Estimator:                 BetweenOLS   R-squared (Between):              0.5443
No. Observations:                 595   R-squared (Within):               0.0762
Date:                Fri, Apr 15 2022   R-squared (Overall):              0.4173
Time:                        14:52:49   Log-likelihood                   -56.142
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      57.926
Entities:                         595   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                  F(12,582)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             57.926
                            

In [7]:
################################################################ 
# Calculando la data promedio por individuo 
# ################################################################ 
data_mean = data.groupby(level=0).mean() 
display(data_mean.head())


,EXP,WKS,OCC,IND,SOUTH,SMSA,MS,FEM,UNION,ED,BLK,LWAGE,EXP^2
Individual,,,,,,,,,,,,,
1,6.0,37.571429,0.000000,0.428571,1.0,0.000000,1.000000,0.0,0.000000,9.0,0.0,5.964759,40.0
2,33.0,31.571429,1.000000,0.714286,0.0,0.000000,1.000000,0.0,0.142857,11.0,0.0,6.498446,1093.0
3,9.0,50.428571,1.000000,1.000000,0.0,0.000000,0.714286,0.0,1.000000,12.0,0.0,6.510670,85.0
4,34.0,47.857143,1.000000,0.000000,0.0,1.000000,0.000000,1.0,0.000000,10.0,1.0,6.386966,1160.0
5,13.0,47.000000,0.571429,0.000000,0.0,0.142857,1.000000,0.0,0.428571,16.0,0.0,6.904354,173.0


#### B.- STATSMODELS

In [8]:
################################################################ 
#Calculando los coeficientes usando Between OLS y Statsmodels
 ################################################################ 
#Definiendo las variables exog = ['EXP', 'EXP^2', 'WKS', 'OCC', 'IND', 'SOUTH', 'SMSA', 'MS', 'FEM', 'UNION', 'ED', 'BLK'] 
Y_mean = data_mean['LWAGE'] 
X_mean = sm.add_constant(data_mean[exog]) 
#Estimando el modelo 
res_between_2 = sm.OLS(Y_mean, X_mean).fit() 
print(res_between_2.summary())


                            OLS Regression Results                            
Dep. Variable:                  LWAGE   R-squared:                       0.544
Model:                            OLS   Adj. R-squared:                  0.535
Method:                 Least Squares   F-statistic:                     57.93
Date:                Fri, 15 Apr 2022   Prob (F-statistic):           4.24e-91
Time:                        14:52:55   Log-Likelihood:                -56.142
No. Observations:                 595   AIC:                             138.3
Df Residuals:                     582   BIC:                             195.3
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.1214      0.204     25.074      0.0

### 12.3.3. MODELO Primeras Diferencias (First Difference)  - DOS METODOS

Este caso consiste en utilizar el estimador MCO sobre las primeras diferencias de las variables a lo largo del tiempo. 

#### A.- LINEARMODELS

In [9]:
######################################################################## 
# Calculando los coeficientes usando Primeras Diferencias y Linearmodels 
######################################################################## #Definiendo las variables 
exog_dif = ['EXP', 'EXP^2', 'WKS', 'OCC', 'IND', 'SOUTH', 'SMSA', 'MS', 'UNION'] 
#Estimando el modelo 
res_first_dif = lm.panel.FirstDifferenceOLS(Y, X[exog_dif]).fit() 
print(res_first_dif)


                     FirstDifferenceOLS Estimation Summary                      
Dep. Variable:                  LWAGE   R-squared:                        0.2247
Estimator:         FirstDifferenceOLS   R-squared (Between):              0.4766
No. Observations:                3570   R-squared (Within):               0.6570
Date:                Fri, Apr 15 2022   R-squared (Overall):              0.4768
Time:                        14:53:00   Log-likelihood                    1035.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      114.68
Entities:                         595   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                  F(9,3561)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             114.68
                            

In [10]:
################################################################ 
#Calculando la data de primeras diferencias por individuo 
################################################################ 
data_diff = data.groupby(level=0).diff().dropna() 
display(data_diff.head())


EXP   WKS  OCC  IND  SOUTH  SMSA   MS  FEM  UNION   ED  BLK  \
Individual Year                                                                
1          2     1.0  11.0  0.0  0.0    0.0   0.0  0.0  0.0    0.0  0.0  0.0   
           3     1.0  -3.0  0.0  0.0    0.0   0.0  0.0  0.0    0.0  0.0  0.0   
           4     1.0  -1.0  0.0  0.0    0.0   0.0  0.0  0.0    0.0  0.0  0.0   
           5     1.0   3.0  0.0  1.0    0.0   0.0  0.0  0.0    0.0  0.0  0.0   
           6     1.0  -7.0  0.0  0.0    0.0   0.0  0.0  0.0    0.0  0.0  0.0   

                   LWAGE  EXP^2  
Individual Year                  
1          2     0.15963    7.0  
           3     0.27614    9.0  
           4     0.00000   11.0  
           5     0.06501   13.0  
           6     0.11233   15.0

In [11]:
################################################################ 
#Depurando las variables que no son linealmente independientes 
################################################################ 
depen = {} 
for i in data_diff.columns.tolist(): 
    depen[i] = 0 
    for j in data_diff.columns.tolist(): 
        if i != j: 
            inner_product = np.inner(data_diff[i],data_diff[j]) 
            norm_i = np.linalg.norm(data_diff[i]) 
            norm_j = np.linalg.norm(data_diff[j]) 

            if np.abs(inner_product - norm_j * norm_i) < 1E-5: 
                depen[i] += 1 
                
rank = np.linalg.matrix_rank(data_diff) 
included = pd.DataFrame(depen, index=['Dependencies']).T 
included = included[included['Dependencies']<=rank] 
included = included.index.tolist() 
included.remove('LWAGE') 
print(included)


['EXP', 'WKS', 'OCC', 'IND', 'SOUTH', 'SMSA', 'MS', 'UNION', 'EXP^2']


#### B.- STATSMODELS

In [12]:
################################################################ 
#Calculando los coeficientes usando Primeras Diferencias y Statsmodels (CON LAS VARIABLES INDEPENDIENTES) 
################################################################ 
#Definiendo las variables 
Y_diff = data_diff['LWAGE'] 
X_diff = data_diff[included] 
#Estimando el modelo 
res_first_dif_2 = sm.OLS(Y_diff, X_diff).fit() 
print(res_first_dif_2.summary())


                            OLS Regression Results                            
Dep. Variable:                  LWAGE   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     4.034
Date:                Fri, 15 Apr 2022   Prob (F-statistic):           8.75e-05
Time:                        14:53:08   Log-Likelihood:                 1035.7
No. Observations:                3570   AIC:                            -2053.
Df Residuals:                    3561   BIC:                            -1998.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
EXP            0.1164      0.006     18.468      0.0

### 12.3.4. Estimador Intra Grupos (Within OLS) - DOS MÉTODOS

Este caso consiste en utilizar el estimador de MCO sobre las desviaciones respecto a la media de las variables a lo largo del tiempo.

In [13]:
############################################################ 
#Creando la data desviada respecto a la media ############################################################ 
data_dev = data - data.groupby(level=0, axis=0).transform('mean') 
display(data_dev.head())


EXP       WKS  OCC       IND  SOUTH  SMSA   MS  FEM  UNION  \
Individual Year                                                               
1          1    -3.0 -5.571429  0.0 -0.428571    0.0   0.0  0.0  0.0    0.0   
           2    -2.0  5.428571  0.0 -0.428571    0.0   0.0  0.0  0.0    0.0   
           3    -1.0  2.428571  0.0 -0.428571    0.0   0.0  0.0  0.0    0.0   
           4     0.0  1.428571  0.0 -0.428571    0.0   0.0  0.0  0.0    0.0   
           5     1.0  4.428571  0.0  0.571429    0.0   0.0  0.0  0.0    0.0   

                  ED  BLK     LWAGE  EXP^2  
Individual Year                             
1          1     0.0  0.0 -0.404079  -31.0  
           2     0.0  0.0 -0.244449  -24.0  
           3     0.0  0.0  0.031691  -15.0  
           4     0.0  0.0  0.031691   -4.0  
           5     0.0  0.0  0.096701    9.0

In [14]:
################################################################ 
# Depurando las variables que no son linealmente independientes 
################################################################ 
depen = {} 

for i in data_dev.columns.tolist(): 
    depen[i] = 0 
    for j in data_dev.columns.tolist(): 
        if i != j: 
            inner_product = np.inner(data_dev[i],data_dev[j]) 
            norm_i = np.linalg.norm(data_dev[i]) 
            norm_j = np.linalg.norm(data_dev[j]) 

            if np.abs(inner_product - norm_j * norm_i) < 1E-5: 
                depen[i] += 1 

rank = np.linalg.matrix_rank(data_dev) 
included = pd.DataFrame(depen, index=['Dependencies']).T 
included = included[included['Dependencies']<=rank] 
included = included.index.tolist() 
included.remove('LWAGE') 
print(included)


['EXP', 'WKS', 'OCC', 'IND', 'SOUTH', 'SMSA', 'MS', 'UNION', 'EXP^2']


#### A.- LINEARMODELS

In [15]:
################################################################ 
#Calculando los coeficientes usando Within OLS y Linearmodels 
################################################################ 
#Definiendo las variables 
Y_dev = data_dev['LWAGE'] 
X_dev = data_dev[included]
#Estimando el modelo 
res_within = lm.panel.PooledOLS(Y_dev, X_dev).fit() 
print(res_within)


                          PooledOLS Estimation Summary                          
Dep. Variable:                  LWAGE   R-squared:                        0.6581
Estimator:                  PooledOLS   R-squared (Between):             -0.0002
No. Observations:                4165   R-squared (Within):               0.6581
Date:                Fri, Apr 15 2022   R-squared (Overall):              0.6581
Time:                        14:53:22   Log-likelihood                    2262.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      889.03
Entities:                         595   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                  F(9,4156)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             889.03
                            

#### B.- STATSMODEL

In [16]:
################################################################ 
#Calculando los coeficientes usando Within OLS y Statsmodels 
################################################################ 
# Estimando el modelo 
res_within_2 = sm.OLS(Y_dev, X_dev).fit() 
print(res_within_2.summary())


                                 OLS Regression Results                                
Dep. Variable:                  LWAGE   R-squared (uncentered):                   0.658
Model:                            OLS   Adj. R-squared (uncentered):              0.657
Method:                 Least Squares   F-statistic:                              889.0
Date:                Fri, 15 Apr 2022   Prob (F-statistic):                        0.00
Time:                        14:53:26   Log-Likelihood:                          2262.9
No. Observations:                4165   AIC:                                     -4508.
Df Residuals:                    4156   BIC:                                     -4451.
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

## 12.4. Efectos fijos (Fixed Effects - FE)

### 12.4.1. Efectos Fijos One Way (FE One way)

Este modelo se origina cuando asumimos que los efectos individuales Ci pueden estar arbitrariamente relacionados con las variables exógenas.


In [17]:
############################################################ 
# Calculando los coeficientes usando Efectos Fijos One Way con Linearmodels 
############################################################ 
#Estimando el modelo 
res_fixed = lm.panel.PanelOLS(Y, X[included], entity_effects=True).fit() 
print(res_fixed)


                          PanelOLS Estimation Summary                           
Dep. Variable:                  LWAGE   R-squared:                        0.6581
Estimator:                   PanelOLS   R-squared (Between):              0.4930
No. Observations:                4165   R-squared (Within):               0.6581
Date:                Fri, Apr 15 2022   R-squared (Overall):              0.4932
Time:                        14:53:33   Log-likelihood                    2262.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      761.75
Entities:                         595   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                  F(9,3561)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             761.75
                            

In [18]:
############################################################ 
# Mostrando los efectos calculados 
############################################################ 
display(res_fixed.estimated_effects.head())


estimated_effects
Individual Year                   
1          1              5.294189
           2              5.294189
           3              5.294189
           4              5.294189
           5              5.294189

### 12.4.2. Prueba de Hipótesis

In [20]:
############################################################ 
# Calculando la Prueba F de Efectos Fijos basada en SSR 
############################################################ 
import scipy.stats as st
n = len(data.groupby(level=0)) 
t = len(data.groupby(level=1)) 
k = len(included) 
dfn = n - 1 
dfd = n * t - n - k 
F = dfd/ dfn * (res_fixed.rsquared - res_pooled.rsquared) /(1 - res_fixed.rsquared) 
pvalue = 1 - st.f.cdf(F, dfn=dfn, dfd=dfd) 
print('F-stat:', F) 
print('p-value:', np.round(pvalue, 4))


F-stat: 4.025249303830634
p-value: 0.0


In [21]:
############################################################ 
# Calculando la Prueba F de Efectos Fijos basada en R^2 
############################################################ 
import scipy.stats as st 
n = len(data.groupby(level=0)) 
t = len(data.groupby(level=1)) 
k = len(included) 
dfn = n - 1 
dfd = n * t - n - k 
F = dfd/ dfn * (res_pooled.resid_ss - res_fixed.resid_ss)/ res_fixed.resid_ss 
pvalue = 1 - st.f.cdf(F, dfn=dfn, dfd=dfd) 
print('F-stat:', F) 
print('p-value:', np.round(pvalue, 4))


F-stat: 30.933867043938477
p-value: 0.0


Como el p-value de la prueba es cero, se rechaza la hipótesis nula H0 de homogeneidad y se acepta que existen efectos fijos.

### 12.4.3. Efectos fijos Two Way (FE Two Way)

Este modelo se origina cuando asumimos que además de los efectos individuales Ci existen efectos temporales Dj pueden estar arbitrariamente relacionados con las variables exógenas.

In [24]:
data.shape

(4165, 13)

In [23]:
pd.IndexSlice[:300,:4]

(slice(None, 300, None), slice(None, 4, None))

In [22]:
############################################################# 
#Eliminando las 3 ultimas observaciones para cada individuo 
############################################################# 
data1 = data.loc[pd.IndexSlice[:300,:4],:] 
data2 = data.loc[pd.IndexSlice[301:,:],:] 
data1 = pd.concat([data1, data2], axis=0) 
display(data1.head())


EXP  WKS  OCC  IND  SOUTH  SMSA  MS  FEM  UNION  ED  BLK  \
Individual Year                                                             
1          1       3   32    0    0      1     0   1    0      0   9    0   
           2       4   43    0    0      1     0   1    0      0   9    0   
           3       5   40    0    0      1     0   1    0      0   9    0   
           4       6   39    0    0      1     0   1    0      0   9    0   
2          1      30   34    1    0      0     0   1    0      0  11    0   

                   LWAGE  EXP^2  
Individual Year                  
1          1     5.56068      9  
           2     5.72031     16  
           3     5.99645     25  
           4     5.99645     36  
2          1     6.16331    900

In [25]:
############################################################## 
#Calculando los coeficientes usando Efectos Fijos Two Way #y Linearmodels 
############################################################## 
Y1 = data1['LWAGE']
included1 = included.copy() 
included1.remove('EXP') 
X1 = data1[included1] 
mod = lm.panel.PanelOLS(Y1, X1, entity_effects=True, time_effects=True) 
res_fixed_two = mod.fit() 
print(res_fixed_two)


                          PanelOLS Estimation Summary                           
Dep. Variable:                  LWAGE   R-squared:                        0.0305
Estimator:                   PanelOLS   R-squared (Between):             -0.0933
No. Observations:                3265   R-squared (Within):              -0.2133
Date:                Fri, Apr 15 2022   R-squared (Overall):             -0.0934
Time:                        15:11:44   Log-likelihood                    1904.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      10.444
Entities:                         595   P-value                           0.0000
Avg Obs:                       5.4874   Distribution:                  F(8,2656)
Min Obs:                       4.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             10.444
                            

In [26]:
############################################################ 
# Mostrando los efectos calculados 
############################################################ 
display(res_fixed_two.estimated_effects.head())


estimated_effects
Individual Year                   
1          1              5.692525
           2              5.799132
           3              5.944774
           4              6.060910
2          1              6.642761

### 12.4.4. Prueba de Hipótesis

In [28]:
############################################################ 
# Calculando Prueba F de para Efectos Fijos Individuales #y Temporales 
############################################################ 
# Estimando el modelo sin restricciones 
mod = lm.panel.PanelOLS(Y1, X1) 
res_pooled_two = mod.fit() 

#Calculando el estadístico F 
n = len(data.groupby(level=0)) 
t = len(data.groupby(level=1)) 
k = len(included) 
dfn = n + t - 2 
dfd = n * t - n - k + 1 
F = dfd/ dfn * (res_pooled_two.resid_ss - res_fixed_two.resid_ss)/ res_fixed_two.resid_ss 
pvalue = 1 - st.f.cdf(F, dfn=dfn, dfd=dfd)

# Mostrando los resultados 
print('F-stat:', F) 
print('p-value:', np.round(pvalue, 4))


F-stat: 163.51266428765703
p-value: 0.0


Como el p-value de la prueba es 0, se rechaza la hipótesis nula H0 y se acepta la hipótesis alternativa H1 de que existe al menos un efecto temporal o individual.

In [29]:
############################################################ 
#Calculando Prueba F de para Efectos Fijos Individuales 
############################################################ 
#Estimando el modelo con efectos individuales 
mod = lm.panel.PanelOLS(Y1, X1, entity_effects=True) 
res_fixed_ind = mod.fit() 

#Estimando el modelo con efectos temporales 
mod = lm.panel.PanelOLS(Y1, X1, time_effects=True) 
res_fixed_temp = mod.fit()

#Calculando el estadístico F 
dfn = n - 1 
dfd = n * t - n - k + 1 
F = dfd/ dfn * (res_fixed_temp.resid_ss - res_fixed_ind.resid_ss)/ res_fixed_ind.resid_ss 
pvalue = 1 - st.f.cdf(F, dfn=dfn, dfd=dfd) 

#Mostrando los resultados 
print('F-stat:', F) 
print('p-value:', np.round(pvalue, 4))


F-stat: 18.27550789407596
p-value: 0.0


Como el p-value de la prueba es 0, se rechaza la hipótesis nula H0 y se acepta la hipótesis alternativa H1 de que existe al menos un efecto individual. 

In [30]:
############################################################ 
#Calculando Prueba F de para Efectos Fijos Temporales 
############################################################ 
#Calculando el estadístico F 
dfn = t - 1 
dfd = n * t - n - k + 1 

F = dfd/ dfn * (res_fixed_ind.resid_ss - res_fixed_temp.resid_ss)/ res_fixed_temp.resid_ss
pvalue = 1 - st.f.cdf(F, dfn=dfn, dfd=dfd) 

#Mostrando los resultados 
print('F-stat:', F) 
print('p-value:', np.round(pvalue, 4))


F-stat: -446.9964104925532
p-value: 1.0


Como el p-value de la prueba es 1, se acepta la hipótesis nula H0 y se rechaza la hipótesis alternativa H1 de que existe al menos un efecto temporal.

## 12.5. Efectos aleatorios (Random Effects - RE)

Este modelo se utiliza cuando se asume que la heterogeneidad individual no observada µi no está correlacionada con Xi,j.

In [31]:
############################################################ 
#Estimando el modelo usando Efectos Aleatorios #y Linearmodels 
############################################################ 
res_random = lm.panel.RandomEffects(Y, X).fit() 
print(res_random)


                        RandomEffects Estimation Summary                        
Dep. Variable:                  LWAGE   R-squared:                        0.3899
Estimator:              RandomEffects   R-squared (Between):             -0.5217
No. Observations:                4165   R-squared (Within):               0.5018
Date:                Fri, Apr 15 2022   R-squared (Overall):             -0.2440
Time:                        15:12:03   Log-likelihood                    815.70
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      221.15
Entities:                         595   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                 F(12,4152)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             221.15
                            

In [32]:
############################################################### 
#Estimando la matriz Omega con Between OLS 
############################################################### 
s2_e = res_fixed.s2 
SSR_b = res_between.resid_ss 
s2_u1 = np.max([0, SSR_b/ (n-k) - s2_e/ t]) 
theta1 = 1 - np.sqrt(s2_e/(t*s2_u1 + s2_e)) 
Omega = s2_e * np.identity(t) + s2_u1 * np.ones((t,t)) 

display(pd.DataFrame(np.round(np.tril(Omega), 4)).replace(0,''))


,0,1,2,3,4,5,6
0,0.0916,,,,,,
1,0.0685,0.0916,,,,,
2,0.0685,0.0685,0.0916,,,,
3,0.0685,0.0685,0.0685,0.0916,,,
4,0.0685,0.0685,0.0685,0.0685,0.0916,,
5,0.0685,0.0685,0.0685,0.0685,0.0685,0.0916,
6,0.0685,0.0685,0.0685,0.0685,0.0685,0.0685,0.0916


In [33]:
############################################################ 
#Estimando el modelo usando GLS y Statsmodels 
############################################################ 
Sigma = np.kron(np.identity(n), Omega) 
res_gls = sm.GLS(Y, X, sigma=Sigma).fit() 
print(res_gls.summary())


                            GLS Regression Results                            
Dep. Variable:                  LWAGE   R-squared:                       0.389
Model:                            GLS   Adj. R-squared:                  0.388
Method:                 Least Squares   F-statistic:                     220.6
Date:                Fri, 15 Apr 2022   Prob (F-statistic):               0.00
Time:                        15:12:15   Log-Likelihood:                -103.96
No. Observations:                4165   AIC:                             233.9
Df Residuals:                    4152   BIC:                             316.3
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.2680      0.098     43.730      0.0

In [35]:
############################################################ 
#Estimando la matriz Omega con GLS (promedio Pooled OLS) 
############################################################ 
n = len(data.groupby(level=0)) 
e1 = res_pooled.resids 
Omega = 0
 
for i in range(1, n+1): 
    e_ = np.array(e1[i], ndmin=2) 
    Omega += e_.T @ e_ 

Omega = 1/n * Omega 

display(pd.DataFrame(np.round(np.tril(Omega), 4)).replace(0,''))


[[84.34289469 65.94249805 48.87454658 34.6726173  21.88582115  9.03394558
  -3.4470536 ]
 [65.94249805 61.65399854 44.49178794 34.43746035 24.8553512  14.866526
   5.88956226]
 [48.87454658 44.49178794 67.51691288 50.30086743 42.48564834 37.33131157
  34.79568205]
 [34.6726173  34.43746035 50.30086743 62.22801597 48.62541015 47.53294448
  48.91883681]
 [21.88582115 24.8553512  42.48564834 48.62541015 59.97524224 56.96444825
  60.91795627]
 [ 9.03394558 14.866526   37.33131157 47.53294448 56.96444825 74.10816104
  74.90756711]
 [-3.4470536   5.88956226 34.79568205 48.91883681 60.91795627 74.90756711
  96.94046302]]


,0,1,2,3,4,5,6
0,0.1418,,,,,,
1,0.1108,0.1036,,,,,
2,0.0821,0.0748,0.1135,,,,
3,0.0583,0.0579,0.0845,0.1046,,,
4,0.0368,0.0418,0.0714,0.0817,0.1008,,
5,0.0152,0.025,0.0627,0.0799,0.0957,0.1246,
6,-0.0058,0.0099,0.0585,0.0822,0.1024,0.1259,0.1629


In [46]:
############################################################ 
#Estimando el modelo usando GLS y Statsmodels 
############################################################ 
Sigma = np.kron(np.identity(n), Omega) 
res_gls_2 = sm.GLS(Y, X, sigma=Sigma).fit() 
print(res_gls_2.summary())


                            GLS Regression Results                            
Dep. Variable:                  LWAGE   R-squared:                       0.163
Model:                            GLS   Adj. R-squared:                  0.160
Method:                 Least Squares   F-statistic:                     67.20
Date:                Thu, 14 Apr 2022   Prob (F-statistic):          3.49e-150
Time:                        20:43:33   Log-Likelihood:                 888.63
No. Observations:                4165   AIC:                            -1751.
Df Residuals:                    4152   BIC:                            -1669.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.3102      0.078     68.122      0.0

In [47]:
############################################################ 
#Estimando la matriz Omega con Pooled OLS y Efectos Fijos 
############################################################ 
s2_e = res_fixed.s2 
s2_u = np.sum(res_pooled.resids.values ** 2)/ (n * t - X.shape[1]) - s2_e 
Omega = s2_e * np.identity(t) + s2_u * np.ones((t,t)) 

display(pd.DataFrame(np.round(np.tril(Omega), 4)).replace(0,''))


,0,1,2,3,4,5,6
0,0.1221,,,,,,
1,0.0990,0.1221,,,,,
2,0.0990,0.099,0.1221,,,,
3,0.0990,0.099,0.099,0.1221,,,
4,0.0990,0.099,0.099,0.099,0.1221,,
5,0.0990,0.099,0.099,0.099,0.099,0.1221,
6,0.0990,0.099,0.099,0.099,0.099,0.099,0.1221


In [48]:
############################################################ 
# Estimando el modelo usando GLS y Statsmodels
############################################################ 
Sigma = np.kron(np.identity(n), Omega) 
res_gls2 = sm.GLS(Y, X, sigma=Sigma).fit() 
print(res_gls2.summary())


                            GLS Regression Results                            
Dep. Variable:                  LWAGE   R-squared:                       0.427
Model:                            GLS   Adj. R-squared:                  0.425
Method:                 Least Squares   F-statistic:                     257.5
Date:                Thu, 14 Apr 2022   Prob (F-statistic):               0.00
Time:                        20:46:10   Log-Likelihood:                -8.4898
No. Observations:                4165   AIC:                             42.98
Df Residuals:                    4152   BIC:                             125.3
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.0408      0.104     38.804      0.0

### 12.5.2. Prueba de Hipótesis

In [50]:
############################################################ 
# Calculando la prueba LM 
############################################################ 
# Calculando el estadístico 
e = np.array(res_pooled.resids, ndmin=2).T 
D = np.kron(np.identity(n), np.ones((t,1))) 
LM = n * t/ (2 * (t - 1)) * (e.T @ D @ D.T @ e /(e.T @ e) - 1) ** 2 
LM = LM.item() 
pvalue = 1 - st.chi2.cdf(LM, df=1) 

# Mostrando los resultados 
print('LM-stat:', LM) 
print('p-value:', np.round(pvalue, 4))


LM-stat: 3497.0184100087668
p-value: 0.0


In [52]:
############################################################ 
#Calculando la prueba Z2 
############################################################
#Calculando el estadístico 
e = np.array(res_pooled.resids, ndmin=2).T 
f = 0.5 * (np.multiply(D.T @ e, D.T @ e) - D.T @ np.multiply(e, e)) 
z_2 = (np.ones((n, 1)).T @ f) ** 2/ (f.T @ f) 
z_2 = z_2.item() 
pvalue = 1 - st.chi2.cdf(z_2, df=1) 

#Mostrando los resultados 
print('z^2-stat:', z_2) 
print('p-value:', np.round(pvalue, 4))


z^2-stat: 179.66308963874408
p-value: 0.0


En ambos casos se rechaza la hipótesis nula H0 y se acepta que el modelo de efectos aleatorios es adecuado para modelar la data que tenemos disponible.

## 12.6. Estimadores de la Matriz de Covarianzas de β

In [53]:
############################################################ 
#Errores estándar robustos usando MCO y Linearmodels 
############################################################
#Usando los errores estándar de White 
res_pooled_hec = lm.panel.PooledOLS(Y, X).fit(cov_type='heteroskedastic') 
#Usando los errores estándar de Driscoll Kraay 
res_pooled_dk = lm.panel.PooledOLS(Y, X).fit(cov_type='kernel') 

#Comparando los errores estándar e intervalos de confianza 
print('') 
print(res_pooled.summary.tables[1]) 
print(res_pooled_hec.summary.tables[1]) 
print(res_pooled_dk.summary.tables[1])



                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          5.2511     0.0713     73.662     0.0000      5.1114      5.3909
EXP            0.0401     0.0022     18.574     0.0000      0.0359      0.0443
EXP^2         -0.0007  4.744e-05    -14.193     0.0000     -0.0008     -0.0006
WKS            0.0042     0.0011     3.8989     0.0001      0.0021      0.0063
OCC           -0.1400     0.0147    -9.5526     0.0000     -0.1687     -0.1113
IND            0.0468     0.0118     3.9673     0.0001      0.0237      0.0699
SOUTH         -0.0556     0.0125    -4.4414     0.0000     -0.0802     -0.0311
SMSA           0.1517     0.0121     12.567     0.0000      0.1280      0.1753
MS             0.0484     0.0206     2.3555     0.0185      0.0081      0.0888
FEM           -0.3678     0.0251    -14.655     0.0

Se observa que los errores estándar robustos son diferentes a los errores estándar homocedásticos. Esto genera intervalos de confianza que consideran los efectos de los errores no esféricos al momento de contrastar las hipótesis de significancia de los parámetros β. Se observa que en algunos casos los intervalos de confianza se amplían y en otros se reducen.

In [54]:
############################################################ 
# Errores estándar agrupados usando Efectos Fijos #y Linearmodels 
############################################################ 

# Cuando se utiliza el tipo cov_type clustered, 
# se puede especificar los grupos usando una variable 
# dummy de grupos 
res_fixed_clus = lm.panel.PanelOLS(Y, X[included], entity_effects=True)
res_fixed_clus = res_fixed_clus.fit(cov_type='clustered', cluster_entity=True) 

# Comparando los errores estándar e intervalos de confianza 
print(res_fixed.summary.tables[1]) 
print(res_fixed_clus.summary.tables[1])


                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
EXP            0.1132     0.0025     45.814     0.0000      0.1084      0.1181
WKS            0.0008     0.0006     1.3940     0.1634     -0.0003      0.0020
OCC           -0.0215     0.0138    -1.5581     0.1193     -0.0485      0.0055
IND            0.0192     0.0154     1.2437     0.2137     -0.0111      0.0495
SOUTH         -0.0019     0.0343    -0.0543     0.9567     -0.0691      0.0654
SMSA          -0.0425     0.0194    -2.1859     0.0289     -0.0806     -0.0044
MS            -0.0297     0.0190    -1.5659     0.1175     -0.0669      0.0075
UNION          0.0328     0.0149     2.1970     0.0281      0.0035      0.0620
EXP^2         -0.0004  5.459e-05    -7.6629     0.0000     -0.0005     -0.0003
                             Parameter Estimates    

Se observa que los errores estándar agrupados son diferentes a los errores estándar homocedásticos. Esto genera intervalos de confianza que consideran los efectos de la correlación dentro de las agrupaciones al momento de contrastar las hipótesis de significancia de los parámetros β. Se observa que en algunos casos los intervalos de confianza se amplían y en otros se reducen.

In [56]:
se1 = res_fixed.conf_int().loc['EXP',:] 
se2 = res_fixed_clus.conf_int().loc['EXP',:] 

columns = ['Homoskedastic', 'Clustered'] 
table = pd.concat([se1, se2], axis=1) 
table.columns = columns 
display(table.T)


,lower,upper
Homoskedastic,0.108363,0.118053
Clustered,0.105275,0.121142


## 12.7. Prueba de Hausman (FE vs RE)

En la práctica, se necesita de un criterio que nos permita comprobar si el modelo de efectos fijos o el de efectos aleatorios se ajusta mejor a la data con la que estamos trabajando.

In [58]:
############################################################ 
# Calculando la prueba de Hausman 
############################################################ 

# Estimando los errores estándar robustos 
res_fixed_rob = lm.panel.PanelOLS(Y, X[included], entity_effects=True) 
res_fixed_rob = res_fixed_rob.fit(use_lsdv=True, cov_type='robust') 

# Definiendo los parámetros iniciales 
b_fe = res_fixed_rob.params
b_re = res_random.params 
b_bw = res_between.params 
sigma_fe = res_fixed_rob.cov 
sigma_re = res_random.cov 
sigma_bw = res_between.cov 

#Eliminando las variables que no son comunes 
if len(b_fe) <= len(b_re): 
    k = len(b_fe) 
    cols = b_fe.index.tolist() 
    b_fe = np.array(b_fe, ndmin=2).T 
    b_re = np.array(b_re[cols], ndmin=2).T 
    b_bw = np.array(b_bw[cols], ndmin=2).T 
    sigma_fe = sigma_fe.values 
    sigma_re = sigma_re.loc[cols,cols].values 
    sigma_bw = sigma_bw.loc[cols,cols].values 
else: 
    k = len(b_re) 
    cols = b_re.index.tolist() 
    b_fe = np.array(b_fe[cols], ndmin=2).T
    b_re = np.array(b_re, ndmin=2).T 
    b_bw = np.array(b_bt[cols], ndmin=2).T 
    sigma_fe = sigma_fe.loc[cols,cols].values 
    sigma_re = sigma_re.values 
    sigma_bw = sigma_bw.loc[cols,cols].values 

#Definiendo los Psis 
Psi = sigma_fe - sigma_re 
Psi_1 = sigma_fe + sigma_bw 

#Calculando los estadísticos 
H = (b_fe - b_re).T @ np.linalg.inv(Psi) @ (b_fe - b_re) 
H = H.item() 
pvalue = 1- st.chi2.cdf(H, df=k-1) 

H_1 = (b_fe - b_bw).T @ np.linalg.inv(Psi_1) @ (b_fe - b_bw) 
H_1 = H_1.item() 
pvalue_1 = 1 - st.chi2.cdf(H_1, df=k-1) 

#Mostrando los resultados 
print('Hausman Test Original :',H)
print('p-value Original :',pvalue) 
print('Hausman Test Alternativo :',H_1) 
print('p-value Alternativo :',pvalue_1)


Hausman Test Original : 8170.7085868229415
p-value Original : 0.0
Hausman Test Alternativo : 3025.279450384721
p-value Alternativo : 0.0


Ambas pruebas rechazan la hipótesis nula H0 y se acepta que el modelo de efectos fijos es más adecuado para modelar la data que tenemos disponible.